## Импорт основных библиотек 

In [1]:
from bs4 import BeautifulSoup
import requests 
import numpy as np
import pandas as pd 
from tqdm import tqdm 
from functools import reduce

In [2]:
def get_anime_urls(npages = 100):
    h3 = []
#             https://myanimelist.net/topanime.php?limit=100
    path = "https://myanimelist.net/topanime.php?limit=" 

    for i in tqdm(range(0, npages*50 + 1,50)): 
        url = path + str(i)
        html = requests.get(url).text
        soup = BeautifulSoup(html)
        h3.append(soup.find_all("h3", class_ = "hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3")) 
    
    h3_clean = reduce(lambda x, y: x + y, h3)
    
    hrefs = [h3.find("a").get("href") for h3 in h3_clean]
    
    return hrefs

In [3]:
urls_dirty = get_anime_urls()

100%|████████████████████████████████████████| 101/101 [02:53<00:00,  1.72s/it]


In [4]:
urls = urls_dirty

In [5]:
## Получение путей карточку аниме-произведения:
urls

['https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
 'https://myanimelist.net/anime/40028/Shingeki_no_Kyojin__The_Final_Season',
 'https://myanimelist.net/anime/9253/Steins_Gate',
 'https://myanimelist.net/anime/28977/Gintama°',
 'https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011',
 'https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2',
 'https://myanimelist.net/anime/9969/Gintama',
 'https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu',
 'https://myanimelist.net/anime/15417/Gintama__Enchousen',
 'https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season',
 'https://myanimelist.net/anime/39486/Gintama__The_Final',
 'https://myanimelist.net/anime/28851/Koe_no_Katachi',
 'https://myanimelist.net/anime/34096/Gintama',
 'https://myanimelist.net/anime/32281/Kimi_no_Na_wa',
 'https://myanimelist.net/anime/918/Gintama',
 'https://myanimelist.net/anime/4181/Clannad__After_Story',
 'https://myanimelist.net/anime/15335/Gintama_Movie_2__Kanket

In [6]:
print("Количество полученных данных: " + str(len(urls)))

Количество полученных данных: 5050


In [7]:
# episodes = []     ## 
# anime_type = []   - 
# duration = [] 
# year = [] 
# members = [] 
# favorites = []   - 
# genre = []       -
# rating = []      -
# studio = []      -
# source = []

In [8]:
def get_full_info(path):
    sp = BeautifulSoup(requests.get(path).text)
    params = sp.find_all("td", class_ = "borderClass", width = 225)
    temp_list = params[0].text.split("\n")
    for el in temp_list:
        if el.isspace():
            temp_list.remove(el)

    while "" in temp_list:
        temp_list.remove("")

    j = 0  
    while temp_list[j] != "Type:":
        temp_list.remove(temp_list[j])

    temp_list.remove("Statistics")

    j = temp_list.index("Score:")
    temp_list.remove(temp_list[j+3])
    temp_list.remove(temp_list[j+2])
    temp_list.remove("Score:")

    j = temp_list.index("Ranked:")
    temp_list.remove(temp_list[j+3])
    temp_list.remove(temp_list[j+2])
    temp_list.remove("Ranked:")

    j = temp_list.index("Rating:")
    temp_list.remove(temp_list[j+3])
    temp_list.remove(temp_list[j+2])

    keys = [temp_list[i] for i in range(0,len(temp_list) ,2)]
    values = [temp_list[i] for i in range(1,len(temp_list) + 1,2)]

    d = dict(zip(keys, values))
    return d

In [9]:
%%time
from tqdm import tqdm 
from time import sleep
animes = []
for path in tqdm(urls):
    try:
        animes.append(get_full_info(path))
    except:
        print(path)
    sleep(1.0)

100%|████████████████████████████████████| 5050/5050 [3:53:04<00:00,  2.77s/it]

Wall time: 3h 53min 4s


In [10]:
for anime in animes:
    try:
        pd.DataFrame({anime})
    except:
        print(anime)

{'Type:': 'TV', 'Episodes:': '  64', 'Status:': '  Finished Airing', 'Aired:': '  Apr 5, 2009 to Jul 4, 2010', 'Premiered:': 'Spring 2009', 'Broadcast:': '    Sundays at 17:00 (JST)', 'Producers:': 'Aniplex,       Square Enix,       Mainichi Broadcasting System,       Studio Moriken ', 'Licensors:': 'Funimation,       Aniplex of America ', 'Studios:': 'Bones ', 'Source:': '  Manga', 'Genres:': 'ActionAction,     MilitaryMilitary,     AdventureAdventure,     ComedyComedy,     DramaDrama,     MagicMagic,     FantasyFantasy,     ShounenShounen', 'Duration:': '  24 min. per ep.', 'Rating:': '  R - 17+ (violence & profanity)', 'Popularity:': '  #3', 'Members:': '    2,134,701', 'Favorites:': '  175,402'}
{'Type:': 'TV', 'Episodes:': '  16', 'Status:': '  Currently Airing', 'Aired:': '  Dec 7, 2020 to ?', 'Premiered:': 'Winter 2021', 'Broadcast:': '    Mondays at 00:10 (JST)', 'Producers:': 'Production I.G,       Dentsu,       Mainichi Broadcasting System,       Pony Canyon,       Kodansha, 

In [11]:
 for i in range(len(animes)):
    for name in animes[i]:
        animes[i][name] = [reduce(lambda x, y: x+y, list(animes[i][name]))]

In [12]:
animeFrame = [pd.DataFrame(anime) for anime in animes]

In [13]:
anime = pd.concat(animeFrame)

In [14]:
anime

,Type:,Episodes:,Status:,Aired:,Premiered:,Broadcast:,Producers:,Licensors:,Studios:,Source:,Genres:,Duration:,Rating:,Popularity:,Members:,Favorites:
0,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Bro...","Funimation, Aniplex of America",Bones,Manga,"ActionAction, MilitaryMilitary, Advent...",24 min. per ep.,R - 17+ (violence & profanity),#3,"2,134,701","175,402"
0,TV,16,Currently Airing,"Dec 7, 2020 to ?",Winter 2021,Mondays at 00:10 (JST),"Production I.G, Dentsu, Mainichi B...",Funimation,MAPPA,Manga,"ActionAction, MilitaryMilitary, Myster...",23 min. per ep.,R - 17+ (violence & profanity),#226,"493,739","19,153"
0,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Mov...",Funimation,White Fox,Visual novel,"ThrillerThriller, Sci-FiSci-Fi",24 min. per ep.,PG-13 - Teens 13 or older,#9,"1,712,236","145,255"
0,TV,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,Manga,"ActionAction, ComedyComedy, Historical...",24 min. per ep.,PG-13 - Teens 13 or older,#329,"387,115","11,426"
0,TV,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,Sundays at 10:55 (JST),"VAP, Nippon Television Network, Sh...",VIZ Media,Madhouse,Manga,"ActionAction, AdventureAdventure, Fant...",23 min. per ep.,PG-13 - Teens 13 or older,#12,"1,576,539","140,372"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Movie,1,Finished Airing,"Jan 3, 2010",NaN,NaN,"None found, add some","None found, add some",Studio Ghibli,Original,ComedyComedy,13 min.,G - All Ages,#8827,"1,933",4
0,TV,44,Finished Airing,"Apr 1, 1976 to Jan 27, 1977",Spring 1976,Unknown,Dynamic Planning,Discotek Media,Toei Animation,Original,"AdventureAdventure, MechaMecha, Sci-Fi...",24 min. per ep.,PG-13 - Teens 13 or older,#9000,"1,815",4
0,OVA,6,Finished Airing,"Jan 21, 1994 to Jun 21, 1994",NaN,NaN,"VAP, Studio Jack","ADV Films, Nozomi Entertainment",Sunrise,Light novel,"ActionAction, AdventureAdventure, Poli...",27 min. per ep.,PG-13 - Teens 13 or older,#5574,"7,081",21
0,TV,31,Finished Airing,"Mar 9, 1988 to Dec 23, 1988",Spring 1988,Unknown,"Kitty Films, Fuji TV","None found, add some",Studio Deen,Manga,"CarsCars, ShounenShounen, SportsSports",25 min. per ep.,PG-13 - Teens 13 or older,#8463,"2,260",11


In [15]:
tun = anime["Type:"].unique()
config = dict(zip(tun, range(len(tun))))

In [16]:
config

{'TV': 0, 'OVA': 1, 'Movie': 2, 'ONA': 3, 'Special': 4, '  Music': 5}

In [17]:
anime = anime.replace(config)
anime

,Type:,Episodes:,Status:,Aired:,Premiered:,Broadcast:,Producers:,Licensors:,Studios:,Source:,Genres:,Duration:,Rating:,Popularity:,Members:,Favorites:
0,0,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Bro...","Funimation, Aniplex of America",Bones,Manga,"ActionAction, MilitaryMilitary, Advent...",24 min. per ep.,R - 17+ (violence & profanity),#3,"2,134,701","175,402"
0,0,16,Currently Airing,"Dec 7, 2020 to ?",Winter 2021,Mondays at 00:10 (JST),"Production I.G, Dentsu, Mainichi B...",Funimation,MAPPA,Manga,"ActionAction, MilitaryMilitary, Myster...",23 min. per ep.,R - 17+ (violence & profanity),#226,"493,739","19,153"
0,0,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Mov...",Funimation,White Fox,Visual novel,"ThrillerThriller, Sci-FiSci-Fi",24 min. per ep.,PG-13 - Teens 13 or older,#9,"1,712,236","145,255"
0,0,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,Manga,"ActionAction, ComedyComedy, Historical...",24 min. per ep.,PG-13 - Teens 13 or older,#329,"387,115","11,426"
0,0,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,Sundays at 10:55 (JST),"VAP, Nippon Television Network, Sh...",VIZ Media,Madhouse,Manga,"ActionAction, AdventureAdventure, Fant...",23 min. per ep.,PG-13 - Teens 13 or older,#12,"1,576,539","140,372"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2,1,Finished Airing,"Jan 3, 2010",NaN,NaN,"None found, add some","None found, add some",Studio Ghibli,Original,ComedyComedy,13 min.,G - All Ages,#8827,"1,933",4
0,0,44,Finished Airing,"Apr 1, 1976 to Jan 27, 1977",Spring 1976,Unknown,Dynamic Planning,Discotek Media,Toei Animation,Original,"AdventureAdventure, MechaMecha, Sci-Fi...",24 min. per ep.,PG-13 - Teens 13 or older,#9000,"1,815",4
0,1,6,Finished Airing,"Jan 21, 1994 to Jun 21, 1994",NaN,NaN,"VAP, Studio Jack","ADV Films, Nozomi Entertainment",Sunrise,Light novel,"ActionAction, AdventureAdventure, Poli...",27 min. per ep.,PG-13 - Teens 13 or older,#5574,"7,081",21
0,0,31,Finished Airing,"Mar 9, 1988 to Dec 23, 1988",Spring 1988,Unknown,"Kitty Films, Fuji TV","None found, add some",Studio Deen,Manga,"CarsCars, ShounenShounen, SportsSports",25 min. per ep.,PG-13 - Teens 13 or older,#8463,"2,260",11


In [21]:
anime["Source:"].unique()    ## 5

array(['  Manga', '  Visual novel', '  Novel', '  Original',
       '  Light novel', '  Web manga', '  Other', '  Game', 5,
       '  4-koma manga', '  Unknown', '  Book', '  Picture book',
       '  Card game', '  Digital manga'], dtype=object)

In [19]:
anime.to_csv("anime.csv")

In [24]:
anime["Type:"].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [27]:
anime["Episodes:"].unique()    ## Unknown

array(['  64', '  16', '  24', '  51', '  148', '  10', '  110', '  13',
       '  22', '  1', '  12', '  201', '  7', '  25', '  14', '  26',
       '  75', '  74', '  4', '  43', '  27', '  47', '  11', '  Unknown',
       '  37', '  39', '  101', '  23', '  99', '  15', '  6', '  120',
       '  112', '  2', '  50', '  62', '  8', '  33', '  40', '  3',
       '  94', '  79', '  77', '  52', '  203', '  96', '  291', '  500',
       '  70', '  38', '  78', '  9', '  28', '  5', '  63', '  103',
       '  45', '  21', '  102', '  153', '  109', '  145', '  30', '  69',
       '  36', '  60', '  34', '  178', '  220', '  35', '  167', '  49',
       '  113', '  366', '  17', '  155', '  54', '  114', '  104',
       '  161', '  373', '  97', '  48', '  31', '  42', '  358', '  61',
       '  175', '  195', '  1787', '  19', '  46', '  20', '  100',
       '  142', '  72', '  127', '  73', '  147', '  150', '  29', '  53',
       '  115', '  76', '  124', '  224', '  18', '  140', '  4

In [28]:
anime["Status:"].unique()

array(['  Finished Airing', '  Currently Airing'], dtype=object)

In [30]:
[el for el in anime["Aired:"].unique()]

['  Apr 5, 2009 to Jul 4, 2010',
 '  Dec 7, 2020 to ?',
 '  Apr 6, 2011 to Sep 14, 2011',
 '  Apr 8, 2015 to Mar 30, 2016',
 '  Oct 2, 2011 to Sep 24, 2014',
 '  Apr 29, 2019 to Jul 1, 2019',
 '  Apr 4, 2011 to Mar 26, 2012',
 '  Jan 8, 1988 to Mar 17, 1997',
 '  Oct 4, 2012 to Mar 28, 2013',
 '  Oct 14, 2017 to Mar 31, 2018',
 '  Jan 8, 2021',
 '  Sep 17, 2016',
 '  Jan 9, 2017 to Mar 27, 2017',
 '  Aug 26, 2016',
 '  Apr 4, 2006 to Mar 25, 2010',
 '  Oct 3, 2008 to Mar 27, 2009',
 '  Jul 6, 2013',
 '  Aug 12, 2017 to Aug 13, 2017',
 '  Apr 6, 2008 to Sep 28, 2008',
 '  Oct 8, 2016 to Dec 10, 2016',
 '  Jul 9, 2018 to Oct 8, 2018',
 '  Jan 7, 2019 to Apr 1, 2019',
 '  Jul 20, 2001',
 '  Aug 15, 2020',
 '  Jan 6, 2017',
 '  Jan 8, 2018 to Mar 26, 2018',
 '  Apr 3, 1998 to Apr 24, 1999',
 '  Jan 7, 2017 to Mar 25, 2017',
 '  Jul 7, 2013 to Dec 29, 2013',
 '  Oct 19, 2014 to Dec 21, 2014',
 '  Apr 11, 2020 to Jun 27, 2020',
 '  Jan 17, 2020',
 '  Oct 10, 2014 to Mar 20, 2015',
 '  Oct 4,

In [31]:
anime = anime.dropna()

In [32]:
anime

,Type:,Episodes:,Status:,Aired:,Premiered:,Broadcast:,Producers:,Licensors:,Studios:,Source:,Genres:,Duration:,Rating:,Popularity:,Members:,Favorites:
0,0,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Bro...","Funimation, Aniplex of America",Bones,Manga,"ActionAction, MilitaryMilitary, Advent...",24 min. per ep.,R - 17+ (violence & profanity),#3,"2,134,701","175,402"
0,0,16,Currently Airing,"Dec 7, 2020 to ?",Winter 2021,Mondays at 00:10 (JST),"Production I.G, Dentsu, Mainichi B...",Funimation,MAPPA,Manga,"ActionAction, MilitaryMilitary, Myster...",23 min. per ep.,R - 17+ (violence & profanity),#226,"493,739","19,153"
0,0,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Mov...",Funimation,White Fox,Visual novel,"ThrillerThriller, Sci-FiSci-Fi",24 min. per ep.,PG-13 - Teens 13 or older,#9,"1,712,236","145,255"
0,0,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,Manga,"ActionAction, ComedyComedy, Historical...",24 min. per ep.,PG-13 - Teens 13 or older,#329,"387,115","11,426"
0,0,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,Sundays at 10:55 (JST),"VAP, Nippon Television Network, Sh...",VIZ Media,Madhouse,Manga,"ActionAction, AdventureAdventure, Fant...",23 min. per ep.,PG-13 - Teens 13 or older,#12,"1,576,539","140,372"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,103,Finished Airing,"Oct 17, 1989 to Sep 29, 1992",Fall 1989,Tuesdays at Unknown,TBS,"None found, add some",Shin-Ei Animation,Manga,"ComedyComedy, PsychologicalPsychological, ...",10 min. per ep.,R+ - Mild Nudity,#6851,"4,239",28
0,0,52,Finished Airing,"Apr 13, 1970 to Apr 5, 1971",Spring 1970,Unknown,"TV Tokyo, DOGA Productions","None found, add some","None found, add some",Unknown,SportsSports,25 min. per ep.,PG-13 - Teens 13 or older,#9284,"1,619",6
0,0,39,Finished Airing,"Apr 5, 2002 to Dec 27, 2002",Spring 2002,Fridays at 18:00 (JST),"TV Tokyo, Victor Entertainment","None found, add some",Nippon Animation,Original,"Sci-FiSci-Fi, AdventureAdventure, Mech...",21 min. per ep.,PG - Children,#8487,"2,227",12
0,0,44,Finished Airing,"Apr 1, 1976 to Jan 27, 1977",Spring 1976,Unknown,Dynamic Planning,Discotek Media,Toei Animation,Original,"AdventureAdventure, MechaMecha, Sci-Fi...",24 min. per ep.,PG-13 - Teens 13 or older,#9000,"1,815",4
